In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
import scipy.integrate as sint

In [2]:
def k1(T):
    T_eV = T / 11605.
    log_T_eV = np.log(T_eV)
    rv = np.exp(-32.71396786375
          + 13.53655609057*log_T_eV
          - 5.739328757388*log_T_eV**2 
          + 1.563154982022*log_T_eV**3
          - 0.2877056004391*log_T_eV**4
          + 0.03482559773736999*log_T_eV**5
          - 0.00263197617559*log_T_eV**6
          + 0.0001119543953861*log_T_eV**7
          - 2.039149852002e-6*log_T_eV**8)
    return rv

def k2(T):
    rv = 4.881357e-6*T**(-1.5)* (1.+1.14813e2 * T**(-0.407))**(-2.242)
    return rv

In [3]:
def rhs(t, state):
    dnHdt = state[1]*state[2]*k2(state[3]) - state[0]*state[2]*k1(state[3])
    dnHpdt = state[0]*state[2]*k1(state[3]) - state[1]*state[2]*k2(state[3])
    dnemdt = state[0]*state[2]*k1(state[3]) - state[1]*state[2]*k2(state[3])
    dTdt = 0
    return np.array([
        dnHdt, dnHpdt, dnemdt, dTdt
    ])

In [4]:
@ipywidgets.interact(n_total = (-3.0, 9.0), e_frac = (-8.0, 0.0),
                     T = (0., 6.), final_t = (0.0, 9.0),
                    safety_factor = (1, 10000))
def evolve(n_total = 2, e_frac = -4, T = np.log10(15000),
           final_t = 7,
           safety_factor = 100):
    final_t = 10**final_t * 365*24*3600
    n_H_initial = 10**n_total * (1.0 - 10**e_frac)
    n_Hp_initial = 10**n_total * 10**e_frac
    n_em_initial = 10**n_total * 10**e_frac
    state_vector = np.array([n_H_initial, n_Hp_initial,  n_em_initial, 10**T])

    integrator = sint.ode(rhs)
    integrator.set_initial_value(state_vector, t=0)
    state_vector_values = []
    ts = []
    dt = final_t / safety_factor
    ts.append(integrator.t)
    state_vector_values.append(integrator.y)
    while integrator.t < final_t:
        integrator.integrate(integrator.t + dt)
        ts.append(integrator.t)
        state_vector_values.append(integrator.y)
    state_vector_values = np.array(state_vector_values)
    ts = np.array(ts)
    plt.loglog(ts, state_vector_values[:,0], label='HI')
    plt.loglog(ts, state_vector_values[:,1], label='HII')
    plt.xlabel("Time [s]")
    plt.ylabel("n")
    plt.legend()

interactive(children=(FloatSlider(value=2.0, description='n_total', max=9.0, min=-3.0), FloatSlider(value=-4.0, description='e_frac', max=0.0, min=-8.0), FloatSlider(value=4.176091259055681, description='T', max=6.0), FloatSlider(value=7.0, description='final_t', max=9.0), IntSlider(value=100, description='safety_factor', max=10000, min=1), Output()), _dom_classes=('widget-interact',))